In [ ]:
from pydub import AudioSegment
import numpy as np
import pandas as pd

song0 = AudioSegment.from_mp3("data/never_gonna_give_you_up.mp3")
# 以毫秒为单位
ten_seconds = 10 * 1000
song=song0[:ten_seconds]
song

In [ ]:
#get_array_of_samples()类型为'array.array' object
song_data=np.array(song.get_array_of_samples())
pd.Series(song_data).plot()

In [ ]:
import cufflinks as cf
def my_fft(b,t):
    N=len(b)
    s = np.fft.fft(b)
    f=np.linspace(0,N/t,N)[0:int(N/2)]
    A=(2*np.sqrt(s[:].imag**2+s[:].real**2)[0:int(N/2)])/N
    return pd.Series(A,index=f)

res=my_fft(song_data,10)
res.iplot()

In [ ]:
res[:1000].iplot()

$$m=2595\log_{10}\left(1+\frac{f}{700}\right)

In [ ]:
#使用梅尔刻度,m是对应的梅尔频度
temp=res
temp.index=2595*np.log10(1+(res.index/700))
temp.iplot()

In [ ]:
inv=np.abs(np.fft.ifft(my_fft(song_data,10)))

#if the audio has multiple channels, the samples for each channel will be serialized
#for example, stereo audio would look like [sample_1_L, sample_1_R, sample_2_L, sample_2_R, …]
#左右声道交错
segment = AudioSegment(data=song_data.astype("float16").tobytes(), sample_width=4, frame_rate=44100, channels=1)
segment
# song0.export("data/never gonna give you up 2.wav" ,format="wav", bitrate="192k")

In [ ]:

import io
import scipy.io.wavfile
def song2arr(sound,frame_rate=44100):
    sound = sound.set_frame_rate(frame_rate)
    channel_sounds = sound.split_to_mono()#得到声道
    samples = [s.get_array_of_samples() for s in channel_sounds]
    return samples

def arr2song(arr,frame_rate=44100):
    fp_arr = np.array(arr).T.astype(np.float32)
    fp_arr /= np.iinfo(arr[0].typecode).max #获得量化值(多少bit)归一化
    wav_io = io.BytesIO()
    scipy.io.wavfile.write(wav_io,frame_rate, fp_arr)
    wav_io.seek(0)#回到文件开头
    sound = AudioSegment.from_wav(wav_io)
    return sound
samples=song2arr(song0)
song1=arr2song(samples)
song1

In [ ]:
np.array(samples).shape

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2)
ax[0].plot(samples[0])
ax[1].plot(samples[1])#左右有区别

In [ ]:
#多声道的傅里叶变换    
t=song0.duration_seconds   
N=len(samples[0])
f=np.linspace(0,N/t,N)
def full_fft(samples,t):
    N=len(samples[0])#采样数
    res=[]
    for channel in samples:
        s = np.fft.fft(channel)
        A=2*np.sqrt(s[:].imag**2+s[:].real**2)/N
        res.append([A,s])
    return res
pd.Series(full_fft(samples,t)[0][0],index=f).plot()

In [ ]:
ifft_res=[]

import array
for channel in full_fft(samples,t):
    s=np.fft.ifft(channel[1])#real为浮点np.int16将其转为整数,便于后续转换
    ifft_res.append(array.array(song0.array_type,np.int16(np.round(s[:].real))))#转回array对象,记得设置量化精度type
    
arr2song(ifft_res)#转回来了啊啊啊

In [ ]:
temp=song2arr(song)
t=song.duration_seconds
fft_res=full_fft(temp,t)
fm,fM=1500,15000
M=1e7
for channel in fft_res:
    length=len(fft_res[0][0])
    channel[1][:].real[fm:fM]=M
    channel[1][:].real[length-fM:length-fm]=M
    channel[1][:].imag[fm:fM]=M
    channel[1][:].imag[length-fM:length-fm]=M
pd.Series(fft_res[0][1].real).iplot()


In [ ]:
ifft_res=[]
for channel in fft_res:
    s=np.fft.ifft(channel[1])#real为浮点np.int16将其转为整数,便于后续转换
    ifft_res.append(array.array(song0.array_type,np.int16(np.round(s[:].real))))#转回array对象,记得设置量化精度type
    
arr2song(ifft_res)